# NenaParser: A parser for Nena Standard Text format

The goal of this parser (under development) is to translate texts in the
[Nena Standard Text format][nenamd] (we should find a better name for that)
into structured groups of morphemes. Those structured morphemes can then be
easily converted to (e.g.) TextFabric format.

For the Nena Standard Text parser, we make use of [Sly][sly], a Python
implementation of the lex/yacc type of parser generators. (This may soon have
to be converted to Sly's predecessor [Ply][ply], as Sly works only with
Python 3.6+ and the NENA website runs on Python 3.5 - but that should not be
difficult).

[nenamd]: https://github.com/CambridgeSemiticsLab/nena_corpus/blob/tomarkdown/docs/text_markup.md
[sly]: https://sly.readthedocs.io/en/latest/
[ply]: http://www.dabeaz.com/ply/index.html

In [1]:
import re

## Lexer

The parser needs as its input 'tokens', which are predefined units of characters. These are provided by the 'lexer'. In Sly (and Ply), tokens are defined as regular expressions, of which the matching string is returned as the token value. If the token is defined as a function (with its regular expression as argument to the `@_` decorator), then the returned value (among other things) can be manipulated. For more detailed information, [see the documentation][slydocs].

[slydocs]: https://sly.readthedocs.io/en/latest/sly.html

In [2]:
from sly import Lexer

punct = '.,?!:;–\u02c8\u2014\u2019\u2018'

class NenaLexer(Lexer):
    
    # set of token names
    tokens = {
        TITLE, ATTRIBUTE, LETTER, NEWLINES, 
        NEWLINE,
        SPACE, PUNCTUATION, HYPHEN,
        LPAREN_COMMENT, LBRACKET_COMMENT, DIGITS,
        LANG_MARKER, COMMENT, FOOTNOTE
    }
    
    # NB \u207A == superscript +
    literals = {'*', '(', ')', '{', '}', '[', ']', '/', '^'}

    # The '(?m)' part turns on multiline matching, which makes
    # it possible to use ^ and $ for the start/end of the line.
    # Title starts with pound sign. Returns 2-tuple (key, value).
    @_(r'(?m)^\# .*$')
    def TITLE(self, t):
        t.value = ('title', t.value[2:])
        return t

    # Attribute starts key and colon. Returns 2-tuple (key, value).
    @_(r'(?m)^[a-z][a-z0-9_]+: .*$')
    def ATTRIBUTE(self, t):
        t.value = tuple(t.value.split(': '))
        return t
    
    # Footnote starts with '[^n]: ', where n is a number.
    # Returns a 2-tuple (int: fn_sym, str: footnote_text)
    @_(r'(?m)^\[\^[1-9][0-9]*\]: \D*$')
    def FOOTNOTE(self, t):
        fn_sym, footnote = t.value.split(maxsplit=1)
        t.value = (int(fn_sym[2:-2]), footnote)
        return t

    # Punctuation is any normal punctuation symbol and vertical bar.
    # as well as a long hyphen (—)
    
    PUNCTUATION = f'[{punct}]'
    
    # How to get combined Unicode characters to be recognized?
    # Matching only Unicode points of letters with pre-combined
    # marks can be done with the 'word' class '\w', but it
    # includes digits and underscore. To remove those, negate
    # the inverted word class along with digits and underscore:
    # '[^\W\d_]. But that does not include separate combining
    # marks, or the '+' sign.
    # One solution would be unicodedata.normalize('NFC', data),
    # except that not all combinations have pre-combined Unicode
    # points.
    # Another solution is to use an external regex engine such as
    # `regex` (`pip install regex`), which has better Unicode
    # support. However, I would like to avoid extra dependencies.
    # Another (less elegant) solution is to make the '+' symbol
    # and the combining characters [\u0300-\u036F] each its own
    # token, which the parser will have to parse into morphemes
    # and words.
    # Another (also less elegant) solution is to use a 'negative
    # lookbehind assertion' for the negation of digits and '_':
    # https://stackoverflow.com/a/12349464/9230612
    # (?!\d_)[\w\u0300-\u036F]+
    # Because combining marks can never appear before the first
    # letter, and because some dialects have a '+' sign at the
    # beginning of some words, we prefix an optional '+' symbol
    # and an obligatory '[^\W\d_]' before the negative lookbehind.
    
    # One letter with (or without) combining marks can be matched
    # with: [^\W\d_][\u0300-\u036F]*
    # We also add a superscript plus (U-207A) as part of a letter, 
    # since this char is not a letter on its own, but rather
    # modifies the quality of a consonant
    # PUNCTUATION is also excluded
    LETTER = f'[\u207A]?[^\W\d_{punct}][\u0300-\u036F]*'
    
    # we try to make a LETTERS token:
#     LETTERS = r'[+]?[^\W\d_](?!\d_)[\w\u0300-\u036F+]*'
    # Unfortunately, with python's `re` it seems impossible to repeat
    # a group like this. So we will group the letters in the parser.
    
    # Newlines: boundaries of paragraphs and metadata are marked
    # with two newlines (meaning an empty line). The empty line
    # may contain whitespace.
    NEWLINES = r'\n\s*\n\s*'
        
    # Space is any successive number of whitespace symbols.
    SPACE = r'\s+'
    # One or more digits, not starting with zero
    DIGITS = r'[1-9][0-9]*'
    # Line id is any number of digits surrounded by round brackets
#     LINE_ID = r'\([0-9]+\)'  # TODO convert to int?
    # There are two different hyphens, a single one and a double one.
    # The double one is the 'equals' sign.
    HYPHEN = r'[-=]'
    # Language markers are ASCII letter strings surrounded by
    # angle brackets.
    LANG_MARKER = r'<[A-Za-z]+>'
    # A special comment starts with an opening bracket, capital initials
    # and a colon.
    LPAREN_COMMENT = r'\([A-Za-z]+:'
    LBRACKET_COMMENT = r'\[[A-Za-z]+:'
    # A regular comment is text (at least one character not being a digit)
    # which may not contain a colon (otherwise it becomes a special comment/interruption)
    COMMENT = r'\([^:)]*[^:)\d]+[^:)]*\)'

lexer_test = """
# Gozáli and Nozali

text_id: A8
informant: Nanəs Bənyamən
place: ʾƐn-Nune

(1) a-\u207Atest word...[^1] (a-comment) [GK: lalala fd] bla //
\u2018blatwo\u2019


(2) also[^2] <E>*wórds*<E>.i.ˈ [GK:
b-mú bəcnàšəva?] b-mù
(3) more wordsčx /
new paragraph (4) new-paragraph — pause. t-wéwa ... ʾáyya  <R>*tséntr*<R>-ət

[^1]: The 
[^2]: Second footnote.
continued.
[^3]: Third footnote, not referenced in text.
"""


# demonstration of output results of lexer, to be used by parser below
lexer = NenaLexer()
[(tok.type, tok.value) for tok in lexer.tokenize(lexer_test)]

[('SPACE', '\n'),
 ('TITLE', ('title', 'Gozáli and Nozali')),
 ('NEWLINES', '\n\n'),
 ('ATTRIBUTE', ('text_id', 'A8')),
 ('SPACE', '\n'),
 ('ATTRIBUTE', ('informant', 'Nanəs Bənyamən')),
 ('SPACE', '\n'),
 ('ATTRIBUTE', ('place', 'ʾƐn-Nune')),
 ('NEWLINES', '\n\n'),
 ('(', '('),
 ('DIGITS', '1'),
 (')', ')'),
 ('SPACE', ' '),
 ('LETTER', 'a'),
 ('HYPHEN', '-'),
 ('LETTER', '⁺t'),
 ('LETTER', 'e'),
 ('LETTER', 's'),
 ('LETTER', 't'),
 ('SPACE', ' '),
 ('LETTER', 'w'),
 ('LETTER', 'o'),
 ('LETTER', 'r'),
 ('LETTER', 'd'),
 ('PUNCTUATION', '.'),
 ('PUNCTUATION', '.'),
 ('PUNCTUATION', '.'),
 ('[', '['),
 ('^', '^'),
 ('DIGITS', '1'),
 (']', ']'),
 ('SPACE', ' '),
 ('COMMENT', '(a-comment)'),
 ('SPACE', ' '),
 ('LBRACKET_COMMENT', '[GK:'),
 ('SPACE', ' '),
 ('LETTER', 'l'),
 ('LETTER', 'a'),
 ('LETTER', 'l'),
 ('LETTER', 'a'),
 ('LETTER', 'l'),
 ('LETTER', 'a'),
 ('SPACE', ' '),
 ('LETTER', 'f'),
 ('LETTER', 'd'),
 (']', ']'),
 ('SPACE', ' '),
 ('LETTER', 'b'),
 ('LETTER', 'l'),
 ('LETTER'

In [1]:
gold_coins = """\
# A Hundred Gold Coins

source: bar text a1-A7.html
text_id: A6
informant: Yuwəl Yuḥanna
place: Dure

(1) xá-ga xèta,ˈ mállah Naṣràdin,ˈ xázəx mòdi wíða.ˈ gu-bɛ̀θa wéwa,ˈ har-zála-w
θàya.ˈ zála-w θàya,ˈ mára ya-ʾàlaha,ˈ yawə̀tliˈ ʾə́mma dàwe.ˈ ʾən-hàwaˈ ʾə́č̣č̣i-u
ʾə́č̣č̣a maqəlbə̀nna.ˈ ʾu-ʾən-hàwaˈ ʾə́mma-w-xà-ži,ˈ la-băyə̀nna.ˈ de-šùqla.ˈ ʾə̀mma
gắrəg háwa drə́st.ˈ (2) b-álaha hóle zála-w θàya,ˈ ʾíθwale xá-šwawa huðàya,ˈ
maṣóθe ʾə́lle dìye.ˈ mə́re xázəx ʾáwwa dū̀s-ile.ˈ qɛ́mən mjarbə̀nne.ˈ síqa l-gàre,ˈ
də́ryɛle ʾə́č̣č̣i-u ʾə́č̣č̣a dáwe gu-ða-kìsta,ˈ də́rya b-kàwele.ˈ ʾá báxta hàyyo!ˈ hóle
ʾaláha qəm-mšadə̀rrən.ˈ (3) muθɛ́θɛla màjma.ˈ msúrqəlla píšela mnáyəlla l-xà-xa.ˈ
plíṭla ʾə́č̣č̣i-u ʾə̀č̣č̣a.ˈ trè,ˈ trè,ˈ ʾə́č̣č̣i-u ʾə̀č̣č̣a.ˈ ʾə̀ṣra,ˈ ʾə̀ṣra,ˈ hàr-ʾəč̣č̣i-u
ʾə́č̣č̣a.ˈ klèla,ˈ ʾámər báxta dū̀s-ile.ˈ ʾaláha là-xaləṭ.ˈ ʾə́č̣č̣i-u ʾə̀č̣č̣a,ˈ ʾáxči
ʾána max-xšàwti,ˈ ʾáyya kìstaˈ hóle mxožə́bnəlla max-xà.ˈ ha-šqùl,ˈ máttula
tămàha.ˈ (4) huðáya l-gàreˈ šwirɛ́le l-pàlga,ˈ yába ʾànən mšúdrəlla!ˈ ʾáy kálba
ʾámər táma l-gàre maṣyóθe,ˈ bắyət šaqlə́tla ʾap-ʾànna.ˈ mrázgət gànux.ˈ tə́mməl
ṱ-ásqəx kəs-qàzi.ˈ huðáya ʾə́č̣č̣i-u ʾə́č̣č̣a dáwe zìle mə́nne,ˈ ʾɛ́ka ṱ-áθya šə̀nθe?!ˈ
hal-qedámta šə́nθe la-θèla.ˈ hár-wele zála-w θàya.ˈ (5) málla múttəlle réše
dmìxa.ˈ ṭlìya,ˈ kéfe basìmta,ˈ ʾu-dáwe xo-rèše.ˈ sáʾət ʾə́šta mbàdlaˈ ʾə́θyɛle
huðáya wáða ṭəq-ṭəq-ṭə́q l-ṭằra.ˈ ʾu-qáre l-tằraˈ mòdila qə́ṣṣət?ˈ (6) pθíxəlle
tắra màlla,ˈ ʾína huðáya lwíša kášxa-w júlle xàθe.ˈ ʾu-xmárte díye msúrgəlla-w
wíðəlla tàza.ˈ ṭla-mòdila ʾáyya?ˈ mə́re ṱ-ásqəx kəs-qàzi.ˈ mə́re sì kálba.ˈ ʾána
là-ʾaθən mə́nnux.ˈ ṭla-mò la-ʾáθət mə́nni?ˈ sí là-ʾaθən mə́nnux ʾána.ˈ (7) ṭla-mò?ˈ
mòdila qə́ṣṣət?ˈ mə́re ʾaláha-w náše yằði.ˈ xazɛ́la ʾánna júllux hàtxa,ˈ kášxa-w
ʾáyya xmàrtuxˈ ʾu-sɛ̀rga-wˈ w-ána b-ánna dašdàše-wˈ b-ánna čak̭àlle-wˈ
ʾu-šəxtàna-wˈ šárṭ qázi t-yawə́lla ṭlàlux.ˈ (8) qázi ṱ-awə́dla ṭlàlux,ˈ lá-ʾawədla
ṭlàli.ˈ mə́re là-ʾaθən,ˈ ʾína júllux hálla ṭlàli.ˈ ʾu-xmártux ṱ-átwən ʾàna
l-xáṣa.ˈ ʾu-ʾáti luš-jùlliˈ ʾu-ṱ-àsqəx.ˈ hám-ʾən ʾàsqətˈ ʾap-ʾáyya qabū̀l-ila.ˈ
(9) qímɛle ʾaw-lwíša dašdàšət málla,ˈ ʾu-čak̭àllət málla,ˈ málla lwíšɛle kášxa
d-o-huðàyaˈ ʾu-tíwɛle xáṣət xmàrta,ˈ ʾu-síqela kəs-qàzi.ˈ síqɛle kəs-qàzi,ˈ
wírela šarṭ-qàzi.ˈ mòdila qə́ṣṣət?ˈ (10) málla mə̀reˈ qázi ṱ-áwət basìma.ˈ ʾána
kəmà dána ṱ-in-mṣalóye ṭla-márya ʾàlahaˈ ta-t-yawə́lli ʾə̀mma dáwe.ˈ ʾu-ʾáwwa
huðáya wéle maṣyóθe nṭára ʾə́lli l-gàre.ˈ ʾu-ʾálaha šuxa-l-šə́mme qəm-mšadə́rri
ʾə́č̣č̣i-u ʾə́č̣č̣a dàwe.ˈ ʾu-ʾána mə́ri bàxta,ˈ ʾáp ʾayya-kìstaˈ ʾálaha mxožə́bnəlla
m-gēb-xà-dawa.ˈ (11) ʾu-ʾáwwa-ži huðáya l-gàreˈ ʾə̀lla,ˈ ʾánna dáwe ʾànən
hiwə́llux.ˈ ʾu-hátxa be-bằxət-ile,ˈ yá-qazi ṱ-áwət basìma,ˈ ʾáwwa huðàya,ˈ yáʿni
díya gu-pàθuxˈ mdàgəlˈ ʾu-sàru.ˈ mə́re xzí hátxa be-nxə̀pθɛle.ˈ lá ʾítle nxə̀pθa.ˈ
(12) díya ṱ-àmərˈ ʾáp ʾanna-júlle ṱ-ilà-lluxˈ ʾu-ʾáyya xmárta dìyila.ˈ w-áti
xə́zyətli ʾána tíwa l-xàṣa.ˈ huðáya mə̀reˈ dìyila!ˈ ʾína d-ɛ̀nila?ˈ qázi mə́re qùˈ
kálba brət-kálba huðàya.ˈ ʾámər sí šarṭ-dìyuxile.ˈ"""

## Parsing a `.nena` file

Below is a representation of the tree-like structure of a NENA standard text file. This is the structure that the parser must recognize and reproduce.

```
text
  |
  heading
  |  |
  |  attributes
  |    |
  |    attribute (e.g. title, informant, etc.)
  | 
  paragraphs
    |
    paragraph
    |  |
    |  lines
    |    |
    |    line
    |      |
    |      line elements (in any order)
    |        |
    |        stress group
    |        |  |
    |        |  word normal (+metadata, e.g. trailer, etc.)
    |        |  |
    |        |  word foreign (+metadata)
    |        |  |
    |        |  word language (+metadata)
    |        |    |
    |        |    letter
    |        |
    |        footnote
    |        |
    |        comment
    |        |
    |        interruption
    |      
    orphaned footnote (processed later)
```

### Morpheme class

To conveniently store the morpheme and its features, we prepare a small `Morpheme` class, to be used by the parser.

In [4]:
class Morpheme:
    
    def __init__(self, value, trailer='',
                 footnotes=None, speaker=None,
                 foreign=False, lang=None):
        self.value = value  # list of (combined) characters
        self.trailer = trailer  # str (TODO: make this a list as well?)
        self.footnotes = footnotes if footnotes is not None else {}  # dict
        self.speaker = speaker  # str
        self.foreign = foreign  # boolean
        self.lang = lang  # str
    
    def __str__(self):
        return ''.join(self.value)
    
    def __repr__(self):
        sp = f' speaker {self.speaker!r}' if self.speaker else ''
        fr = ' foreign' if self.foreign else ''
        ln = f' lang {self.lang!r}' if self.lang else ''
        fn = f' fn_anc {",".join(str(n) for n in self.footnotes)!r}' if self.footnotes else ''
        fn = f' fn_anc {self.footnotes!r}' if self.footnotes else ''
        return f'<Morpheme {str(self)!r} trailer {self.trailer!r}{sp}{fr}{ln}{fn}>'

### The parser

The parser processes the tokens provided by the lexer, and tries to combine them into structured units. Those units are defined in the methods of the `NenaParser` class, with the patterns passed as arguments to the `@_` decorator.

The top unit (in this case, `text`) is returned as the result of the parsing, and in this case contains a tuple `(heading, paragraphs)`.

The value `heading` contains a dictionary with the text metadata. The value `paragraphs` is a list, in which each element contains a list of `lines`. Each element of `lines` is a 2-tuple containing an `int` line identifier, and a list of `line_elements`. The values of `line_elements` are `Morpheme` objects, or 2-tuples with comments in the form `('comment', str)`.

In [5]:
from sly import Parser

# dict stack to contain footnote anchors,
# until the corresponding footnote is encountered.
fn_anchors = {}

class NenaParser(Parser):
    
    debugfile = 'parser.out'

    # Get the token list from the lexer (required)
    tokens = NenaLexer.tokens
    
    def error(self, t):
        #print('ERROR:')
        #print(f'\tunexpected string {repr(t.value[0])} at index {t.index}')
        raise Exception(f'unexpected string {repr(t.value[0])} at index {t.index}')
    
    @_('heading NEWLINES paragraphs')
    def text(self, p):
        return (p.heading, p.paragraphs)
    
    # -- HEADING --
    
    @_('SPACE TITLE NEWLINES attributes',
       'TITLE NEWLINES attributes')
    def heading(self, p):
        key, value = p.TITLE
        heading = {key: value}
        heading.update(p.attributes)
        return heading
    
    @_('attributes space ATTRIBUTE')
    def attributes(self, p):
        key, value = p.ATTRIBUTE
        p.attributes[key] = value
        return p.attributes 
    
    @_('ATTRIBUTE')
    def attributes(self, p):
        key, value = p.ATTRIBUTE
        return {key: value}
    
    # -- PARAGRAPHS --
    
    @_('paragraphs NEWLINES paragraph')
    def paragraphs(self, p):
        # handle cases of null footnotes
        if p.paragraph is not None:
            return p.paragraphs + [p.paragraph]
        else:
            return p.paragraphs
        
    @_('paragraph')
    def paragraphs(self, p):
        return [p.paragraph]
    
    # paragraph
    @_('paragraph line')
    def paragraph(self, p):
        return p.paragraph + [p.line]
    
    # paragraph from orphaned footnotes
    @_('footnotes')
    def paragraph(self, p):
        if p.footnotes:
            # TODO: issue log warning about
            # unreferenced footnotes?
            return ('footnotes', p.footnotes)
    
    # -- FOOTNOTES -- 
    
    @_('footnotes footnote')
    def footnotes(self, p):
        p.footnotes.update(p.footnote)
        return p.footnotes
    
    @_('footnote')
    def footnotes(self, p):
        return p.footnote
    
    @_('FOOTNOTE space NEWLINES',
       'FOOTNOTE NEWLINES',
       'FOOTNOTE space',
       'FOOTNOTE')
    def footnote(self, p):
        fn_sym, fn_str = p.FOOTNOTE
        footnote = {}
        try:
            # lookup the fn_sym key in the fn_anchors dict,
            # and add the footnote to the appropriate morpheme
            fn_morpheme = fn_anchors.pop(fn_sym)
            fn_morpheme.footnotes[fn_sym] = fn_str
        except KeyError:
            # This means there is not footnote anchor
            # referring to this footnote. So we return
            # the footnote to the text
            footnote = {fn_sym: fn_str}
        return footnote

    # -- LINES --
    
    @_('line')
    def paragraph(self, p):
        return [p.line]
    
    @_('line_id line_elements')
    def line(self, p):
        return (p.line_id, p.line_elements)
    
    @_('"(" DIGITS ")" SPACE')
    def line_id(self, p):
        return int(p.DIGITS)

    @_('line_elements line_element',
       'line_element')
    def line_elements(self, p):
        if len(p) == 2:
            return p.line_elements + p.line_element
        else:
            return p.line_element
    
    @_('morphemes',
       'fn_anchor',
       'interruption',
       'morphemes_foreign',
       'morphemes_language',
       'comment')
    def line_element(self, p):
        return p[0]
    
    # -- MORPHEMES -- 

    # morphemes_language
    @_('lang morphemes_foreign morpheme_trailer lang trailer',
       'lang morphemes_foreign lang trailer',
       'lang morphemes_foreign lang',
       'lang morphemes_foreign')
    def morphemes_language(self, p):
        # check if language markers correspond
        if len(p) > 2:
            lang = p.lang0
            if p.lang0 != p.lang1:
                pass  # TODO issue warning: language markers do not correspond
        else:
            lang = p.lang  # TODO issue warning: missing second language marker
        for m in p.morphemes_foreign:
            m.lang = lang
        if len(p) == 4:
            p.morphemes_foreign[-1].trailer += p.trailer
        elif len(p) == 5:
            p.morpheme_trailer.trailer += p.trailer
            p.morphemes_foreign.append(morpheme_trailer)
        return p.morphemes_foreign
    
    # lang
    @_('LANG_MARKER')
    def lang(self, p):
        return p.LANG_MARKER[1:-1]

    # morphemes_foreign
    # last morpheme may not include trailer
    # add trailer after second asterisk to last morpheme
    @_('"*" morphemes letters "*" trailer',
       '"*" morphemes letters "*"',
       '"*" letters "*" trailer',
       '"*" letters "*"',
      )
    def morphemes_foreign(self, p):
        try:
            trailer = p.trailer
        except KeyError:
            trailer = ''
        try:
            morphemes = p.morphemes
        except KeyError:
            morphemes = []
        morphemes.append(Morpheme(p.letters, trailer=trailer))
        for m in morphemes:
            m.foreign = True
        return morphemes
    
    # comment
    @_('COMMENT trailer',
       'COMMENT')
    def comment(self, p):
        return [('comment', p.COMMENT[1:-1])]

    # interruption
    @_('LPAREN_COMMENT space morphemes ")" trailer',
       'LPAREN_COMMENT space morphemes ")"',
       'LBRACKET_COMMENT space morphemes "]" trailer',
       'LBRACKET_COMMENT space morphemes "]"')
    def interruption(self, p):
        speaker = p[0][1:-1]
        for m in p.morphemes:
            m.speaker = speaker
        try:
            trailer = p.trailer
            if (p.morphemes[-1].trailer.endswith(' ')
                and trailer.startswith(' ')):
                trailer = trailer[1:]
            p.morphemes[-1].trailer += trailer
        except KeyError:
            pass
        return p.morphemes
    
    # morphemes
    @_('morphemes morpheme_trailer',
       'morpheme_trailer')
    def morphemes(self, p):
        if len(p) == 2:
            return p.morphemes + [p.morpheme_trailer]
        else:
            return [p.morpheme_trailer]
    
    # -- MORPHEME ATTRIBUTES --
    
    # morpheme_trailer
    @_('letters trailer',
       'letters')
    def morpheme_trailer(self, p):
        if len(p) == 2:
            trailer = p[1]
        else:
            trailer = ''
        return Morpheme(p.letters, trailer=trailer)

    # morpheme_trailer with footnote anchor
    @_('morpheme_trailer fn_anchor trailer',
       'morpheme_trailer fn_anchor')
    def morpheme_trailer(self, p):
        if len(p) == 3:
            if (p.morpheme_trailer.trailer.endswith(' ')
                and p.trailer.startswith(' ')):
                p.trailer = p.trailer[1:]
            p.morpheme_trailer.trailer += p.trailer
        # add dummy value {fn_anc: None} to footnote dict
        p.morpheme_trailer.footnotes[p.fn_anchor] = None
        # add morpheme object to fn_anchors dict,
        # for easy access when footnote text is found
        fn_anchors[p.fn_anchor] = p.morpheme_trailer
        return p.morpheme_trailer
    
    # --VARIOUS--
    
    @_('"[" "^" DIGITS "]"')
    def fn_anchor(self, p):
        return int(p.DIGITS)
        
    @_('letters LETTER')
    def letters(self, p):
        return p.letters + [p.LETTER]
    
    @_('LETTER')
    def letters(self, p):
        return [p[0]]
    
    # trailer
    @_('trailer versebreak',
       'trailer linebreak',
       'trailer PUNCTUATION',
       'trailer space',
       'PUNCTUATION',
       'space',
       'HYPHEN',
      )
    def trailer(self, p):
        return ''.join(p)
    
    # -- LITERALS --
    
    # reduce any number of spaces (\s+)
    # to a single space (' ')
    @_('SPACE')
    def space(self, p):
        return ' '
    
    @_('"/" "/"',
       '"/" "/" space',
       '"/" "/" NEWLINES',
       '"/" "/" space NEWLINES')
    def versebreak(self, p):
        return '//'
    
    @_('"/"',
       '"/" space',
       '"/" NEWLINES',
       '"/" space NEWLINES')
    def linebreak(self, p):
        return '/'
    
parser_test = """
# Gozáli and Nozali

text_id: A8
informant: Nanəs Bənyamən
place: ʾƐn-Nune

(1) a-\u207Atest word...[^1] (a-comment) [GK: lalala fd] bla //
\u2018blatwo\u2019


(2) also[^2] <E>*wórds*<E>.i.ˈ [GK:
b-mú bəcnàšəva?] b-mù
(3) more wordsčx /
new paragraph (4) new-paragraph — pause. t-wéwa ... ʾáyya  <R>*tséntr*<R>-ət

[^1]: The 
[^2]: Second footnote.
continued.
[^3]: Third footnote, not referenced in text.

"""

# demonstration of output results of parser, to be used by generate_TF loop
parser = NenaParser()
parser.parse(lexer.tokenize(parser_test))

Parser debugging for NenaParser written to parser.out


({'title': 'Gozáli and Nozali',
  'text_id': 'A8',
  'informant': 'Nanəs Bənyamən',
  'place': 'ʾƐn-Nune'},
 [[(1,
    [<Morpheme 'a' trailer '-'>,
     <Morpheme '⁺test' trailer ' '>,
     <Morpheme 'word' trailer '... ' fn_anc {1: 'The '}>,
     ('comment', 'a-comment'),
     <Morpheme 'lalala' trailer ' ' speaker 'GK'>,
     <Morpheme 'fd' trailer ' ' speaker 'GK'>,
     <Morpheme 'bla' trailer ' //‘'>,
     <Morpheme 'blatwo' trailer '’'>])],
  [(2,
    [<Morpheme 'also' trailer ' ' fn_anc {2: 'Second footnote.\ncontinued.'}>,
     <Morpheme 'wórds' trailer '.' foreign lang 'E'>,
     <Morpheme 'i' trailer '.ˈ '>,
     <Morpheme 'b' trailer '-' speaker 'GK'>,
     <Morpheme 'mú' trailer ' ' speaker 'GK'>,
     <Morpheme 'bəcnàšəva' trailer '? ' speaker 'GK'>,
     <Morpheme 'b' trailer '-'>,
     <Morpheme 'mù' trailer ' '>]),
   (3,
    [<Morpheme 'more' trailer ' '>,
     <Morpheme 'wordsčx' trailer ' /'>,
     <Morpheme 'new' trailer ' '>,
     <Morpheme 'paragraph' trailer ' '>

### Parser output

The parser prints a warning that there were shift/reduce conflicts, probably caused by ambiguous whitespace. That is not a problem (although not very elegant, ideally it should be fixed). The parser resolves the conflicts automatically.

The output of the example text shows that the parser succeeded to parse it, and structure it into heading, paragraphs, lines and morphemes, with the features stored in the Morpheme object.

## Testing with Real Texts

In [12]:
from pathlib import Path

In [13]:
# paths
data_dir = Path('../nena/0.01')
dialect_dirs = list(Path(data_dir).glob('*'))

### Run Parse On All Texts

In [17]:
name2parsed = {}
name2text = {}
not_parsed = []

ignore = [
    #'The Adventures Of Two Brothers.nena', # FIX BY MOVING UNEMPHASIZED OUT
]

for dialect in dialect_dirs:
    print(f'--Dialect {dialect}--')
    print()
    for file in sorted(dialect.glob('*.nena')):
        
        if file.name in ignore:
            print('SKIPPING:', file.name, '\n')
            not_parsed.append(file)
            continue
        
        with open(file, 'r') as infile:
            text = infile.read()
            name2text[file.name] = text
            print(f'trying: {file.name}')
            parseit = parser.parse(lexer.tokenize(text))
            print(f'\t√')
            name2parsed[file.name] = parseit
                
print(len(name2parsed), 'parsed...')
print(len(not_parsed), 'not parsed...')

--Dialect ../nena/0.01/corrections.sh--

--Dialect ../nena/0.01/Barwar--

trying: A Hundred Gold Coins.nena
	√
trying: A Man Called Čuxo.nena
	√
trying: A Tale Of A Prince And A Princess.nena
	√
trying: A Tale Of Two Kings.nena
	√
trying: Baby Leliθa.nena
	√
trying: Dəmdəma.nena
	√
trying: Gozali And Nozali.nena
	√
trying: I Am Worth The Same As A Blind Wolf.nena
	√
trying: Man Is Treacherous.nena
	√
trying: Measure For Measure.nena
	√
trying: Nanno And Jəndo.nena
	√
trying: Qaṭina Rescues His Nephew From Leliθa.nena
	√
trying: Sour Grapes.nena
	√
trying: Tales From The 1001 Nights.nena
	√
trying: The Battle With Yuwanəs The Armenian.nena
	√
trying: The Bear And The Fox.nena
	√
trying: The Brother Of Giants.nena
	√
trying: The Cat And The Mice.nena
	√
trying: The Cooking Pot.nena
	√
trying: The Crafty Hireling.nena
	√
trying: The Crow And The Cheese.nena
	√
trying: The Daughter Of The King.nena
	√
trying: The Fox And The Lion.nena
	√
trying: The Fox And The Miller.nena
	√
trying: The F

In [18]:
#name2text['A Man Called Čuxo.nena'][7100:7120]

## Evaluation

### Issues (HV)

> HV: The parser does not enforce all parts of the grammar. For example, as verse and line breaks are just appended to the trailer, nothing will stop it from adding other trailer elements (save whitespace) after it. *-HV*

CK: Unsure of what the problem would be here since trailers can have numerous elements. I will keep an eye on this during the TF conversion. Maybe I will see what you mean.

> HV: There is also no check to see whether the two `LANG_MARKER`s have the same value.

CK: This should be dealt with at the source text level as a correction. I will watch for such cases.

> HV: A `+` sign must appear as the first character in a `morpheme` [in Urmi -CK], but that just means that a `+` in the middle of a morpheme breaks it into two morphemes, instead of invalidating it. Undoubtedly there are more issues like this.

I see three such cases:
```
./Urmi_C/The Snake’s Dilemma.nena:pacúrəl k̭at-nùra⁺bəlláyələˈ ʾu-méša bək̭yàdəl.ˈ ʾu-pacúrəl xa-ʾilána ⁺ràmaˈ
./Urmi_C/The Snake’s Dilemma.nena:xlā́p dìyyux,ˈ k̭a⁺tàla márəl,ˈ ʾát m-ìca brílux ⁺ʾal-dá dúca?ˈ m-ìca tpək̭lux
./Urmi_C/The Assyrians Of Urmi.nena:(13) ʾíta ʾé ⁺dánəva tìlunˈ ⁺huyyə́rrun k̭a⁺ʾaturáyə k̭át ⁺p̂àlši.ˈ ⁺ʾaturáyə=zə
```

Since + is treated as a part of the letter (since it functions together with a consonant to convey quality), this problem has been solved. The examples above have been correctly parsed as such (respectively):

```
<Morpheme 'nùra⁺bəlláyələˈ' trailer ' '>
<Morpheme 'k̭a⁺tàla' trailer ' '>
<Morpheme 'k̭a⁺ʾaturáyə' trailer ' '>
```

> HV: Paragraphs in which the first line lacks a `line_id` break the parser. That is true for e.g. the first line of the text in which the `line_id` is absent, or for poetic style text with no `//` verse break marker but with empty line dividing verses. This could (should?) be handled by fixing the issue (default `line_id=1` for first line, default `//` verse break for empty lines within `line`), and issuing a warning notifying the user of the automatic fix.

CK: This is now treated at the source in the corrections script. 

> HV: Footnotes can only be one line and the string is not processed (e.g. markup like `*emphasis*` is kept as is).

CK: The multiline issue is now solved in the parser. I will mark the missing emphasis in issues to be dealt with later. I consider footnotes an extra feature.

> HV: Footnote anchors can now only occur after a `morpheme`, not after other things like `comment` or `interruption`. (note to self: possible solution: include `fn_anc` in `morphemes` instead of `morpheme_trailer`, and put `comment` in `trailer`).

CK: Solved in the parser.

> HV: Comments and unreferenced footnotes are returned as tuples for now, and have to be filtered out in the loop.

This is a good solution for the time being, since the TF conversion script can take it or leave it.

### Questions (HV)

Some questions require answers for implementation. They need not be definitive answers for now, but they should be motivated somehow (even if the motivation is 'random choice'), so it will be clear later why it is done in one way or another.

- How to store hyphen? Now it is stored as a character in a word occuring between morphemes (I think).
  
  Should it be the trailer of the morpheme?

- How to split sentences?

  Now sentences are split on .?! and subsentences on ,
  There are other symbols: ;:– and even .. ... ..., .... ..... (If I recall correctly). Should those split
  sentences or subsentences?


- What to do with poetic line breaks and sentence/paragraph boundaries?

  I think a 'poem' should not be divided into paragraphs. I suggest that a line break '/' is a subsentence division, and a verse break '//' a sentence division (even when in the source it is followed by an empty line). If there is a verse number in between, that automatically starts a new sentence.